In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)

LR_tune,N_value = np.meshgrid(lr_tune,n_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)

lrn_tune = np.hstack((LR_tune,N_value))

In [6]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [7]:
label = "Thinplate_stan"
loss_thresh = 10000
x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))


lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [8]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [9]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [10]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(n_val)

    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [11]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [12]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [13]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [14]:
nan_tune = []
for tune_reps in range(25):
    label = "3D_HTTP_atanh_tune"+str(tune_reps)
    max_reps = 10
    max_iter = 100

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    alpha_full = []
    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    
    n_val = lrn_tune[tune_reps,1] 
    
    
    for reps in range(max_reps):
        print(label)
        train_loss = []
        test_mse_loss = []
        test_re_loss = []
        alpha_val = []


        print(reps)

        torch.manual_seed(reps*36)
        N_D = 5000 #Total number of data points for 'y'
        N_N = 3500
        N_f = 10000 #Total number of collocation points 

        layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
 
        PINN = Sequentialmodel(layers,n_val)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())


        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrn_tune[tune_reps,0], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')



        nan_flag = train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        alpha_full.append(alpha_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full, "label": label}
    savemat(label+'.mat', mdic)

3D_HTTP_atanh_tune0
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 1274342.2 Test MSE 312234.7497242322 Test RE 0.9905581601253657
1 Train Loss 837590.75 Test MSE 298290.21949798166 Test RE 0.9681861574364605
2 Train Loss 706668.25 Test MSE 298861.04636127927 Test RE 0.9691121055224561
3 Train Loss 631501.75 Test MSE 294340.891003

96 Train Loss 179455.95 Test MSE 47857.94240168697 Test RE 0.3878076515200871
97 Train Loss 179237.56 Test MSE 47745.44417450284 Test RE 0.38735157944680965
98 Train Loss 178852.55 Test MSE 47425.51839361393 Test RE 0.38605164340354703
99 Train Loss 178752.05 Test MSE 47413.4354576575 Test RE 0.3860024617120265
Training time: 246.40
3D_HTTP_atanh_tune0
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): L

92 Train Loss 173346.17 Test MSE 45166.177405347546 Test RE 0.37674372736977024
93 Train Loss 173258.97 Test MSE 45065.464131207955 Test RE 0.3763234541494878
94 Train Loss 173161.94 Test MSE 44928.18188348771 Test RE 0.3757498227069684
95 Train Loss 172999.81 Test MSE 44848.21518799109 Test RE 0.37541527930327795
96 Train Loss 172564.27 Test MSE 44795.58992712064 Test RE 0.37519495697956035
97 Train Loss 172104.44 Test MSE 44706.24269658377 Test RE 0.37482059688363845
98 Train Loss 171735.81 Test MSE 44469.906038720095 Test RE 0.37382855164023193
99 Train Loss 171498.12 Test MSE 44172.9473579744 Test RE 0.37257829510001816
Training time: 241.71
3D_HTTP_atanh_tune0
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bi

88 Train Loss 155611.53 Test MSE 39876.00043645397 Test RE 0.3539933945456841
89 Train Loss 155503.03 Test MSE 39755.383861425915 Test RE 0.3534576110289978
90 Train Loss 155287.95 Test MSE 39505.34129444768 Test RE 0.35234431712061604
91 Train Loss 155033.73 Test MSE 39230.53946252163 Test RE 0.3511167130941164
92 Train Loss 154846.72 Test MSE 39061.55873345902 Test RE 0.3503597009776366
93 Train Loss 154701.3 Test MSE 38973.7950076794 Test RE 0.34996588459097694
94 Train Loss 154480.78 Test MSE 38915.721480713335 Test RE 0.34970505125653656
95 Train Loss 154336.48 Test MSE 38893.10308999475 Test RE 0.3496034096212699
96 Train Loss 154142.62 Test MSE 38781.27484526201 Test RE 0.3491004453989583
97 Train Loss 153801.81 Test MSE 38890.30481423705 Test RE 0.34959083278546516
98 Train Loss 153627.39 Test MSE 39008.04876816663 Test RE 0.35011964193975154
99 Train Loss 153359.89 Test MSE 38928.65360953967 Test RE 0.3497631518769396
Training time: 238.32
3D_HTTP_atanh_tune0
3
Sequentialmodel

83 Train Loss 127520.625 Test MSE 31613.769391639376 Test RE 0.31519368137858406
84 Train Loss 127107.26 Test MSE 31381.716599054682 Test RE 0.31403475143273785
85 Train Loss 126323.41 Test MSE 31019.448983469625 Test RE 0.31221689594443197
86 Train Loss 125276.836 Test MSE 30740.3633164085 Test RE 0.31080919623060776
87 Train Loss 124772.766 Test MSE 30404.89576450486 Test RE 0.30910862384298726
88 Train Loss 124313.73 Test MSE 30068.001078939644 Test RE 0.3073913489408243
89 Train Loss 123981.11 Test MSE 29968.799344259736 Test RE 0.3068838501527063
90 Train Loss 123074.92 Test MSE 29689.472018197797 Test RE 0.30545033051785525
91 Train Loss 122188.87 Test MSE 29325.271493588665 Test RE 0.30357107109296155
92 Train Loss 121242.34 Test MSE 28737.915593821766 Test RE 0.30051558142238566
93 Train Loss 119489.5 Test MSE 27453.669610033332 Test RE 0.29372408775056874
94 Train Loss 119261.08 Test MSE 27334.118769892826 Test RE 0.29308385870881837
95 Train Loss 118952.19 Test MSE 27220.5659

...

2 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
3 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
4 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
5 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
6 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
7 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
8 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
9 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
10 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
11 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
12 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
13 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE 1.000323798470912
14 Train Loss 9.696434e+16 Test MSE 318421.568907835 Test RE

0 Train Loss 2.1541587e+16 Test MSE 317893.03216400184 Test RE 0.9994932525641027
1 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
2 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
3 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
4 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
5 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
6 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
7 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
8 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
9 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
10 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
11 Train Loss 2.1283363e+16 Test MSE 317893.0131672372 Test RE 0.999493222700066
12 Train Loss 2.1283363e+16 Test MSE

0 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
1 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
2 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
3 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
4 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
5 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
6 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
7 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
8 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
9 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
10 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
11 Train Loss 9.7851235e+16 Test MSE 318884.036983931 Test RE 1.0010499584043742
12 Train Loss 9.7851235e+16 Test MSE 3

0 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
1 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
2 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
3 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
4 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
5 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
6 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
7 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
8 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
9 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
10 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
11 Train Loss 4.0192527e+16 Test MSE 318936.44553076994 Test RE 1.001132216235013
12 Train Loss 4.0192527e+1

0 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
1 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
2 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
3 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
4 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
5 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
6 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
7 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
8 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
9 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
10 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
11 Train Loss 5.630602e+16 Test MSE 318301.76056837314 Test RE 1.0001355913574905
12 Train Loss 5.630602e+16

In [15]:
import scipy.io as sio

In [16]:
for tune_reps in range(25):
    label = "3D_HTTP_atanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.3736892097350904
1   [[1.0004081  1.00040802 1.00040703 1.00040644 1.00040486 1.00040243
  1.00040064 1.00040001 1.00040203 1.00040376 1.00040965 1.00041425
  1.00041949 1.00042575 1.00043344 1.00044382 1.00045368 1.00047927
  1.00048439 1.00049612 1.00050099 1.00050369 1.0005159  1.0005254
  1.00053852 1.00055902 1.00057388 1.00058859 1.00061012 1.00061559
  1.0006206  1.00062541 1.00063442 1.0006403  1.00063917 1.00064195
  1.00064654 1.00065111 1.00066367 1.00068378 1.00069181 1.00069583
  1.00069727 1.00069877 1.00069872 1.00069898 1.00069912 1.00069868
  1.00069827 1.00069651 1.00069258 1.00069481 1.00069221 1.00068892
  1.00069032 1.00068741 1.0006825  1.00068182 1.00068009 1.00067613
  1.00066545 1.00064257 1.00063282 1.00061877 1.00060102 1.00059069
  1.00058687 1.00058319 1.00058481 1.00058838 1.00057603 1.000561
  1.00055043 1.00055566 1.00056018 1.00056145 1.00056324 1.00056855
  1.00056579 1.00055556 1.00055622 1.00055313 1.00055385 1.00055411
  1.00055524 1.00055596 

In [17]:
lrn_tune[5]

array([0.1, 1. ])